In [5]:
import tensorflow as tf
from tensorflow import keras
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers

In [2]:
batch_size=16
epochs=10
input_shape = (20,21,3)
classes = 6

In [3]:
x,y=[],[]
with h5py.File('dynamic_gestures2.hdf5', 'a') as f:
    print(f.filename)
    for i,gesture in enumerate(list(f['/'])):
        print(i,gesture)
        data = list(f['/'+gesture])
        x.extend(data)
        y.extend([i]*len(data))

x=np.array(x)
y=np.array(y) 
y=tf.keras.utils.to_categorical(y,classes)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

dynamic_gestures2.hdf5
0 flick left
1 flick right
2 point down
3 point left
4 point right
5 point up
(392, 20, 21, 3) (44, 20, 21, 3) (392, 6) (44, 6)


In [18]:
inputs = layers.Input(shape=input_shape)
x = inputs
print(x.shape)
x = layers.TimeDistributed(layers.Flatten())(x)
print(x.shape)
x = layers.TimeDistributed(layers.Dense(32, activation='relu'))(x)
print(x.shape)
x = layers.Flatten()(x)
print(x.shape)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()


(None, 20, 21, 3)
(None, 20, 63)
(None, 20, 32)
(None, 640)
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 32)            2048      
_________________________________________________________________
flatten_10 (Flatten)         (None, 640)               0         
_________________________________________________________________
Output (Dense)               (None, 6)                 3846      
Total params: 5,894
Trainable params: 5,894
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=0.1)
model.evaluate(x_test,y_test)

Epoch 1/10
22/22 [==============================] - 3s 95ms/step - loss: 1.6219 - accuracy: 0.4055 - val_loss: 1.1634 - val_accuracy: 0.7250
Epoch 2/10
22/22 [==============================] - 0s 4ms/step - loss: 0.9542 - accuracy: 0.8380 - val_loss: 0.7844 - val_accuracy: 0.8000
Epoch 3/10
22/22 [==============================] - 0s 5ms/step - loss: 0.5774 - accuracy: 0.8939 - val_loss: 0.5498 - val_accuracy: 0.8250
Epoch 4/10
22/22 [==============================] - 0s 5ms/step - loss: 0.3999 - accuracy: 0.9233 - val_loss: 0.3384 - val_accuracy: 0.9000
Epoch 5/10
22/22 [==============================] - 0s 5ms/step - loss: 0.2642 - accuracy: 0.9572 - val_loss: 0.3022 - val_accuracy: 0.8750
Epoch 6/10
22/22 [==============================] - 0s 4ms/step - loss: 0.1977 - accuracy: 0.9503 - val_loss: 0.2265 - val_accuracy: 0.9250
Epoch 7/10
22/22 [==============================] - 0s 5ms/step - loss: 0.1309 - accuracy: 0.9798 - val_loss: 0.1649 - val_accuracy: 0.9250
Epoch 8/10
22/22 [=

[0.05524594709277153, 1.0]

In [20]:
model.save("test_model_2", save_format = "tf")

INFO:tensorflow:Assets written to: test_model_2\assets
